In [ ]:
import requests
import pandas as pd
import time
import re
import os

# 🔹 Load Excel file and clean column names
df = pd.read_excel("candidates_with_emails.xlsx")
df.columns = df.columns.str.strip().str.lower()  # Remove spaces & lowercase

# 🔹 Check if required columns exist
if "company" not in df.columns or "industry" not in df.columns:
    print("❌ ERROR: Required columns not found in the Excel file.")
    print("Available columns:", df.columns.tolist())
    exit()

# 🔹 Function to clean text (removes problematic characters)
def clean_text(text):
    return re.sub(r"[^\w\s]", "", text)  # Removes special characters (keeps words & spaces)

# 🔹 Function to get company location using OpenStreetMap (Free API)
def get_location_osm(company, industry):
    company = clean_text(company)
    industry = clean_text(industry)
    query = f"{company}, {industry}"
    url = f"https://nominatim.openstreetmap.org/search?format=json&q={query}"

    headers = {
        "User-Agent": "MyCompanyLocationFinder/1.0 (your-email@example.com)"  # Replace with your email
    }

    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        data = response.json()

        if data:
            return {
                "Company": company,
                "Industry": industry,
                "Address": data[0].get("display_name", "Not Found"),
                "Latitude": data[0]["lat"],
                "Longitude": data[0]["lon"],
                "Map Link": f"https://www.openstreetmap.org/?mlat={data[0]['lat']}&mlon={data[0]['lon']}"
            }
        else:
            print(f"⚠️ No result found for: {query}")
            return None  # Try Bing API if OSM fails
    
    except requests.exceptions.RequestException as e:
        print(f"❌ OSM Error fetching data for {company}: {e}")
        return None  # Try Bing API if OSM fails

# 🔹 Function to get company location using Bing Maps API (Backup)
def get_location_bing(company, industry):
    BING_API_KEY = "YOUR_BING_MAPS_API_KEY"  # Replace with your actual key
    query = f"{company}, {industry}"
    url = f"http://dev.virtualearth.net/REST/v1/Locations?query={query}&key={BING_API_KEY}"

    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        data = response.json()

        if data["resourceSets"] and data["resourceSets"][0]["resources"]:
            location = data["resourceSets"][0]["resources"][0]
            return {
                "Company": company,
                "Industry": industry,
                "Address": location["name"],
                "Latitude": location["point"]["coordinates"][0],
                "Longitude": location["point"]["coordinates"][1],
                "Map Link": f"https://www.bing.com/maps?q={location['point']['coordinates'][0]},{location['point']['coordinates'][1]}"
            }
        else:
            print(f"⚠️ No result found in Bing for: {query}")
            return {
                "Company": company,
                "Industry": industry,
                "Address": "Not Found",
                "Latitude": "N/A",
                "Longitude": "N/A",
                "Map Link": "N/A"
            }

    except requests.exceptions.RequestException as e:
        print(f"❌ Bing Error fetching data for {company}: {e}")
        return {
            "Company": company,
            "Industry": industry,
            "Address": "Error",
            "Latitude": "N/A",
            "Longitude": "N/A",
            "Map Link": "N/A"
        }

# 🔹 Store results in a list
locations = []

# 🔹 Search for each company in the Excel file
for _, row in df.iterrows():
    company = str(row["company"]).strip()
    industry = str(row["industry"]).strip()
    
    if company and industry:
        result = get_location_osm(company, industry)  # Try OpenStreetMap
        if result is None:  # If OSM fails, use Bing Maps
            result = get_location_bing(company, industry)
        
        locations.append(result)
        time.sleep(2)  # Wait to avoid API blocks

# 🔹 Save results to a new Excel file
output_file = "company_locations.xlsx"

# 🔹 Check if file is open in Excel
if os.path.exists(output_file):
    try:
        with open(output_file, "r+"):  # Try opening the file in read+write mode
            pass
    except PermissionError:
        print("❌ Error: The file 'company_locations.xlsx' is open in Excel. Please close it and try again.")
        exit()

try:
    df_locations = pd.DataFrame(locations)
    df_locations.to_excel(output_file, index=False)
    print(f"✅ Locations saved successfully to {output_file}!")

except PermissionError:
    print("❌ Error: The file is open in Excel. Please close it and try again.")




⚠️ No result found for: LIS Holding SpA , Servizi IT e consulenza IT
❌ Bing Error fetching data for LIS Holding S.p.A. +: 401 Client Error: Unauthorized for url: http://dev.virtualearth.net/REST/v1/Locations?query=LIS%20Holding%20S.p.A.%20+,%20Servizi%20IT%20e%20consulenza%20IT&key=YOUR_BING_MAPS_API_KEY
⚠️ No result found for: Massilly , Fabbricazione di materiali da imballaggio e contenitori
❌ Bing Error fetching data for Massilly -: 401 Client Error: Unauthorized for url: http://dev.virtualearth.net/REST/v1/Locations?query=Massilly%20-,%20Fabbricazione%20di%20materiali%20da%20imballaggio%20e%20contenitori&key=YOUR_BING_MAPS_API_KEY
⚠️ No result found for: IGT , Sviluppo di software personalizzati per sistemi IT
❌ Bing Error fetching data for IGT -: 401 Client Error: Unauthorized for url: http://dev.virtualearth.net/REST/v1/Locations?query=IGT%20-,%20Sviluppo%20di%20software%20personalizzati%20per%20sistemi%20IT&key=YOUR_BING_MAPS_API_KEY
⚠️ No result found for: CEBAT SpA , Edilizia


: 